This is my notebook following Kaggle's [Getting Started with Python: Kaggle's Titanic Competition](https://www.kaggle.com/c/titanic/details/getting-started-with-python), translating everything into pandas before I realised that [Part 2](https://www.kaggle.com/c/titanic/details/getting-started-with-python-ii) was not a Pandas version, not a simple continuation of part one.

It's a bit of an incomplete mess.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

## Training data

In [2]:
train_df = pd.read_csv('./data/train.csv', header=0)

In [3]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
print(train_df.shape, train_df.size)

(891, 12) 10692


In [6]:
survived_all = train_df[train_df['Survived'] == 1]

In [7]:
print(survived_all.shape[0])

342


In [8]:
round(survived_all.shape[0]/train_df.shape[0], 4)

0.3838

In [9]:
female_passengers = train_df[train_df['Sex'] == 'female']
male_passengers = train_df[train_df['Sex'] == 'male']

In [10]:
female_survived = female_passengers[female_passengers['Survived'] == 1]
male_survived = male_passengers[male_passengers['Survived'] == 1]

In [11]:
print('Percentage female surivors: {:.2%}'.format(female_survived.shape[0]/female_passengers.shape[0]))
print('Percentage male surivors  : {:.2%}'.format(male_survived.shape[0]/male_passengers.shape[0]))

Percentage female surivors: 74.20%
Percentage male surivors  : 18.89%


## Test Data

In [12]:
testdf = pd.read_csv('./data/test.csv', header=0)

In [13]:
testdf.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [14]:
survived_pred_df = testdf[testdf['Sex'] == 'female']
survived_pred_df = survived_pred_df.assign(Survived=np.ma.ones(survived_pred_df.shape[0], dtype=np.bool))

In [15]:
survived_pred_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,True
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,True
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q,True
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C,True
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S,True


In [16]:
survived_pred_df.to_csv('genderbasedmodel.csv')

## Predictions

In [17]:
pred_df = train_df.copy()
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [18]:
pred_df['Age'].mean()

29.69911764705882

In [19]:
pred_df['Fare'] = pred_df['Fare'].apply(np.clip, args=(None, 39))

In [20]:
fair_ceiling = 40
bin_size = 10
num_bins = fair_ceiling//bin_size

In [21]:
num_classes = len(pred_df['Pclass'].unique())
print(num_classes)

3


In [22]:
fare_bin_col = 'FareBin'
pred_df[fare_bin_col] = pred_df['Fare'].floordiv(bin_size)

In [23]:
pred_df[pred_df['Fare'] > 20].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareBin
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,39.0000,C85,C,3.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,39.0000,C123,S,3.0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,39.0000,E46,S,3.0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,2.0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,3.0


In [24]:
col_groups = ('Sex', 'Pclass', fare_bin_col,)
grouped_df = pred_df.groupby(col_groups)

In [25]:
for name, group in grouped_df:
    mean_survived = group['Survived'].mean()
    predict_survival = mean_survived >= 0.5
    group_summary = 'Gender: {} | Class: {} | Fare Bin: {:.0f} | Mean Survived: {:.1%} | Predict Survival: {}'.format(*name, mean_survived, predict_survival)
    group_data = group[['Fare', 'Age', 'Cabin', 'Survived']].head().sort_values('Age')
    print(group_summary, group_data, sep='\n', end='\n\n')

Gender: female | Class: 1 | Fare Bin: 2 | Mean Survived: 83.3% | Predict Survival: True
        Fare   Age Cabin  Survived
136  26.2833  19.0   D47         1
194  27.7208  44.0    B4         1
796  25.9292  49.0   D17         1
177  28.7125  50.0   C49         0
11   26.5500  58.0  C103         1

Gender: female | Class: 1 | Fare Bin: 3 | Mean Survived: 97.7% | Predict Survival: True
    Fare   Age Cabin  Survived
3   39.0  35.0  C123         1
1   39.0  38.0   C85         1
61  39.0  38.0   B28         1
52  39.0  49.0   D33         1
31  39.0   NaN   B78         1

Gender: female | Class: 2 | Fare Bin: 1 | Mean Survived: 91.4% | Predict Survival: True
     Fare   Age Cabin  Survived
84   10.5  17.0   NaN         1
56   10.5  21.0   NaN         1
66   10.5  29.0   F33         1
123  13.0  32.5  E101         1
15   16.0  55.0   NaN         1

Gender: female | Class: 2 | Fare Bin: 2 | Mean Survived: 90.0% | Predict Survival: True
      Fare   Age Cabin  Survived
58   27.75   5.0   NaN  

In [26]:
predict_group_df = grouped_df.apply(lambda x: pd.Series({'PredictSurvival': x['Survived'].mean() >= 0.5}))

In [27]:
prediction_table = predict_group_df.reset_index()

In [28]:
prediction_table

,Sex,Pclass,FareBin,PredictSurvival
0,female,1,2.0,True
1,female,1,3.0,True
2,female,2,1.0,True
3,female,2,2.0,True
4,female,2,3.0,True
5,female,3,0.0,True
6,female,3,1.0,True
7,female,3,2.0,False
8,female,3,3.0,False
9,male,1,0.0,False


In [29]:
norm_df = testdf.copy()
norm_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [30]:
norm_df[norm_df['Name'].str.contains('storey', case=False)]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [31]:
# Fill missing ages with median values
grouped_norm = norm_df.groupby(('Sex', 'Pclass'))
grouped_norm = grouped_norm.apply(lambda x: pd.Series({'AgeFill': x['Age'].median()})).reset_index()
grouped_norm

,Sex,Pclass,AgeFill
0,female,1,41.0
1,female,2,24.0
2,female,3,22.0
3,male,1,42.0
4,male,2,28.0
5,male,3,24.0


In [32]:
norm_df['Fare'] = norm_df['Fare'].apply(np.clip, args=(None, 39))
norm_df[fare_bin_col] = norm_df['Fare'].floordiv(bin_size)

In [33]:
test_pred_df = norm_df.merge(prediction_table, how='left')
test_pred_df.shape

(418, 13)

In [34]:
test_pred_df[test_pred_df[fare_bin_col].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FareBin,PredictSurvival
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,NaN,NaN


In [35]:
test_pred_df[['PassengerId', 'Name', 'Pclass', 'Sex', 'Fare', 'PredictSurvival']].sort_values(['PassengerId', 'Sex'], ascending=True)

,PassengerId,Name,Pclass,Sex,Fare,PredictSurvival
0,892,"Kelly, Mr. James",3,male,7.8292,False
1,893,"Wilkes, Mrs. James (Ellen Needs)",3,female,7.0000,True
2,894,"Myles, Mr. Thomas Francis",2,male,9.6875,False
3,895,"Wirz, Mr. Albert",3,male,8.6625,False
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",3,female,12.2875,True
5,897,"Svensson, Mr. Johan Cervin",3,male,9.2250,False
6,898,"Connolly, Miss. Kate",3,female,7.6292,True
7,899,"Caldwell, Mr. Albert Francis",2,male,29.0000,False
8,900,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",3,female,7.2292,True
9,901,"Davies, Mr. John Samuel",3,male,24.1500,False
